In [1]:
import numpy as np
import pandas as pd
import datetime
import torch.optim as optim
from scipy import io
import argparse
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import h5py  
import matplotlib.pyplot as plt
import matplotlib
# print(matplotlib.__version__)
matplotlib.use('Agg')
import h5py

In [2]:
from PIL import Image
import math
from sklearn.metrics import confusion_matrix
import pylab as pl
import numpy as np
import itertools
import torchvision

In [3]:
current_file_data_save = 'generated_dataset'
data_file = 'singlecoil_dataset'


In [4]:
os.makedirs(current_file_data_save+'/test/', exist_ok=True)  
os.makedirs(current_file_data_save+'/valid/', exist_ok=True)
os.makedirs(current_file_data_save+'/train/', exist_ok=True)
matrix_size = int(256)

In [5]:
# pf_line = int(np.floor(matrix_size*0.45))
pf_line = int(np.floor(matrix_size*0.25))
pf_line_com = matrix_size-pf_line
pf_line_o =int(np.floor(256*0.25)) 
mid = int(matrix_size/2)

SS_flag = 1
MS_flag = 0
EMS_flag = 0

In [6]:
pf_line_o

64

In [7]:
filePath =data_file + '/test/'
filename = os.listdir(filePath)
length = len(filename)
print(length)

1


In [8]:
files = os.listdir(filePath)
for idx, filename in enumerate(files):
    if filename.endswith('.npy'):
        data_path = os.path.join(filePath, filename)
        
        kspace = np.load(data_path)
        
        
        kspace_r = kspace[:,:,:,0]
        kspace_i = kspace[:,:,:,1]

        data = kspace_r + kspace_i *1j
        

        sz = data.shape
        data2 = np.reshape(data,(-1,1,sz[1],sz[2]))

        freq= np.fft.ifftshift(data2 ,axes=(2,3))
        freq = np.fft.ifft2(freq,axes=(2,3))
        img = np.fft.fftshift(freq,axes=(2,3))

        scale_ref = round(np.max(np.absolute(img)),15)
        img = img/scale_ref
 
        img_ref = img        

        freq= np.fft.ifftshift(img_ref ,axes=(2,3))
        freq= np.fft.fft2(freq ,axes=(2,3))
        
        test_k= np.fft.fftshift(freq,axes=(2,3))   
        

        test_data = np.copy(test_k)
        test_data[:,0,:,pf_line:mid] = 0
       
        label = np.zeros((170, 1, 256, 256), dtype=np.complex128) 

        label[:,0,:,:] = test_k[:,0,:,:] - test_data[:,0,:,:]
        label =label.reshape(sz[0],-1,sz[1],sz[2])


        kspaceR = np.copy(np.real(test_data))
        kspaceI = np.copy(np.imag(test_data))

        kspace = np.concatenate((kspaceR,kspaceI), axis=1)


        labelR = np.copy(np.real(label))
        labelI = np.copy(np.imag(label))

        label = np.concatenate((labelR,labelI), axis=1)
        

        for i in range(30,int(sz[0]-30)):
            D = torch.from_numpy(kspace[i,:,:,:]).float()
            L = torch.from_numpy(label[i,:,:,:]).float()
            data = {'k-space':D,'label':L}
            torch.save(data,current_file_data_save +'/test/'+str(idx)+'_'+str(i)+'.pth')


        
        
        print("Processed file", idx, ":", data_path)

Processed file 0 : singlecoil_dataset/test/e13991s3_P01536.7.npy


In [9]:
filePath =data_file + '/valid/'
filename = os.listdir(filePath)
length = len(filename)
print(length)

7


In [10]:
files = os.listdir(filePath)
for idx, filename in enumerate(files):
    if filename.endswith('.npy'):
        data_path = os.path.join(filePath, filename)
        
        kspace = np.load(data_path)
        
        
        kspace_r = kspace[:,:,:,0]
        kspace_i = kspace[:,:,:,1]

        data = kspace_r + kspace_i *1j
        

        sz = data.shape
        data2 = np.reshape(data,(-1,1,sz[1],sz[2]))

        freq= np.fft.ifftshift(data2 ,axes=(2,3))
        freq = np.fft.ifft2(freq,axes=(2,3))
        img = np.fft.fftshift(freq,axes=(2,3))

        scale_ref = round(np.max(np.absolute(img)),15)
        img = img/scale_ref
 
        img_ref = img        

        freq= np.fft.ifftshift(img_ref ,axes=(2,3))
        freq= np.fft.fft2(freq ,axes=(2,3))
        
        test_k= np.fft.fftshift(freq,axes=(2,3))   
        

        test_data = np.copy(test_k)
        test_data[:,0,:,pf_line:mid] = 0
       
        label = np.zeros((sz[0], 1, sz[1], sz[2]), dtype=np.complex128) 

        label[:,0,:,:] = test_k[:,0,:,:] - test_data[:,0,:,:]
        label =label.reshape(sz[0],-1,sz[1],sz[2])


        kspaceR = np.copy(np.real(test_data))
        kspaceI = np.copy(np.imag(test_data))

        kspace = np.concatenate((kspaceR,kspaceI), axis=1)


        labelR = np.copy(np.real(label))
        labelI = np.copy(np.imag(label))

        label = np.concatenate((labelR,labelI), axis=1)
        

        for i in range(30,int(sz[0]-30)):
            D = torch.from_numpy(kspace[i,:,:,:]).float()
            L = torch.from_numpy(label[i,:,:,:]).float()
            data = {'k-space':D,'label':L}
            torch.save(data,current_file_data_save +'/valid/'+str(idx)+'_'+str(i)+'.pth')


        
        
        print("Processed file", idx, ":", data_path)

Processed file 0 : singlecoil_dataset/valid/e14079s3_P09216.7.npy
Processed file 1 : singlecoil_dataset/valid/e14080s3_P18944.7.npy
Processed file 2 : singlecoil_dataset/valid/e13993s4_P16896.7.npy
Processed file 3 : singlecoil_dataset/valid/e14089s3_P53248.7.npy
Processed file 4 : singlecoil_dataset/valid/e13992s4_P08704.7.npy
Processed file 5 : singlecoil_dataset/valid/e14081s3_P25600.7.npy
Processed file 6 : singlecoil_dataset/valid/e14078s3_P02048.7.npy


In [11]:
filePath =data_file + '/train/'
filename = os.listdir(filePath)
length = len(filename)
print(length)

17


In [12]:
files = os.listdir(filePath)
for idx, filename in enumerate(files):
    if filename.endswith('.npy'):
        data_path = os.path.join(filePath, filename)
        
        kspace = np.load(data_path)
        
        
        kspace_r = kspace[:,:,:,0]
        kspace_i = kspace[:,:,:,1]

        data = kspace_r + kspace_i *1j
        

        sz = data.shape
        data2 = np.reshape(data,(-1,1,sz[1],sz[2]))

        freq= np.fft.ifftshift(data2 ,axes=(2,3))
        freq = np.fft.ifft2(freq,axes=(2,3))
        img = np.fft.fftshift(freq,axes=(2,3))

        scale_ref = round(np.max(np.absolute(img)),15)
        img = img/scale_ref
 
        img_ref = img        

        freq= np.fft.ifftshift(img_ref ,axes=(2,3))
        freq= np.fft.fft2(freq ,axes=(2,3))
        
        test_k= np.fft.fftshift(freq,axes=(2,3))   
        

        test_data = np.copy(test_k)
        test_data[:,0,:,pf_line:mid] = 0
       
        label = np.zeros((170, 1, 256, 256), dtype=np.complex128) 

        label[:,0,:,:] = test_k[:,0,:,:] - test_data[:,0,:,:]
        label =label.reshape(sz[0],-1,sz[1],sz[2])


        kspaceR = np.copy(np.real(test_data))
        kspaceI = np.copy(np.imag(test_data))

        kspace = np.concatenate((kspaceR,kspaceI), axis=1)


        labelR = np.copy(np.real(label))
        labelI = np.copy(np.imag(label))

        label = np.concatenate((labelR,labelI), axis=1)
        

        for i in range(30,int(sz[0]-30)):
            D = torch.from_numpy(kspace[i,:,:,:]).float()
            L = torch.from_numpy(label[i,:,:,:]).float()
            data = {'k-space':D,'label':L}
            torch.save(data,current_file_data_save +'/train/'+str(idx)+'_'+str(i)+'.pth')


        
        
        print("Processed file", idx, ":", data_path)

Processed file 0 : singlecoil_dataset/train/e14120s11_P66048.7.npy
Processed file 1 : singlecoil_dataset/train/e14487s5_P47616.7.npy
Processed file 2 : singlecoil_dataset/train/e14423s3_P29696.7.npy
Processed file 3 : singlecoil_dataset/train/e14280s3_P44032.7.npy
Processed file 4 : singlecoil_dataset/train/e14369s5_P40960.7.npy
Processed file 5 : singlecoil_dataset/train/e14292s3_P85504.7.npy
Processed file 6 : singlecoil_dataset/train/e14437s5_P49152.7.npy
Processed file 7 : singlecoil_dataset/train/e14498s5_P60928.7.npy
Processed file 8 : singlecoil_dataset/train/e14477s5_P34816.7.npy
Processed file 9 : singlecoil_dataset/train/e14268s3_P26112.7.npy
Processed file 10 : singlecoil_dataset/train/e14134s3_P06656.7.npy
Processed file 11 : singlecoil_dataset/train/e14141s3_P58880.7.npy
Processed file 12 : singlecoil_dataset/train/e14427s3_P76288.7.npy
Processed file 13 : singlecoil_dataset/train/e14140s3_P52224.7.npy
Processed file 14 : singlecoil_dataset/train/e14110s3_P59904.7.npy
Proc